<a href="https://colab.research.google.com/github/MASAYA-KINOSHITA/first-Repository/blob/main/Software_anomaly_detection_%E7%9B%B8%E9%96%A2%E4%BF%82%E6%95%B026_%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ライブラリのインポート
import pandas as pd # 基本ライブラリ
import numpy as np # 基本ライブラリ
import matplotlib.pyplot as plt # グラフ描画用
import seaborn as sns; sns.set() # グラフ描画用
import warnings # 実行に関係ない警告を無視
warnings.filterwarnings('ignore')

!pip install optuna #optunaのインストール

import optuna.integration.lightgbm as lgb_tune #LightGBMのパラメータチューニング
import lightgbm as lgb #LightGBM
from sklearn import datasets
from sklearn.model_selection import train_test_split # データセット分割用
from sklearn.metrics import accuracy_score # モデル評価用(正答率)
from sklearn.metrics import log_loss # モデル評価用(logloss)     
from sklearn.metrics import roc_auc_score # モデル評価用(auc)
# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error



# データフレームを綺麗に出力する関数
import IPython
def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

     |████████████████████████████████| 308 kB 5.2 MB/s 
     |████████████████████████████████| 210 kB 48.2 MB/s 
     |████████████████████████████████| 81 kB 7.2 MB/s 
     |████████████████████████████████| 78 kB 5.6 MB/s 
     |████████████████████████████████| 113 kB 38.9 MB/s 
     |████████████████████████████████| 49 kB 4.5 MB/s 
     |████████████████████████████████| 150 kB 46.9 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=daba2437a42a2624c0ab5efedebf3608599a529ac89625dcfcdd91136ae6081d
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
#メモリ節約用の関数
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
# data_train csvファイルの読み込み
train_df = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/train.zip (Unzipped Files)/train.csv',usecols=['Anomaly',
 'GC activity : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionCount))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionTime))',
 'Rel. swap usage : ((MXBean(java.lang:type=OperatingSystem).FreeSwapSpaceSize / MXBean(java.lang:type=OperatingSystem).TotalSwapSpaceSize))',
 'Stuck threads : (MXBean(com.bea:Name=ThreadPoolRuntime,Type=ThreadPoolRuntime).StuckThreadCount)',
 'Stuck threads : (MXBean(com.bea:ApplicationRuntime=sys01,Name=default,Type=WorkManagerRuntime).StuckThreadCount)',
 'Process CPU : (MXBean(java.lang:type=OperatingSystem).ProcessCpuLoad)',
 'Rel. heap committed : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.committed / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'System CPU : (MXBean(java.lang:type=OperatingSystem).SystemCpuLoad)',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Process CPU : (\\Process(java)\\CPU)',
 'Rel. heap usage : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.used / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'Rel. open file descriptors : ((MXBean(java.lang:type=OperatingSystem).OpenFileDescriptorCount / MXBean(java.lang:type=OperatingSystem).MaxFileDescriptorCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.committed / MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Daemon thread count : (MXBean(java.lang:type=Threading).DaemonThreadCount)',
 'host',
 'Total thread count : (MXBean(java.lang:type=Threading).ThreadCount)',
 'Rel. nonHeap usage : ((MXBean(java.lang:type=Memory).NonHeapMemoryUsage.used / MXBean(java.lang:type=Memory).NonHeapMemoryUsage.max))',
 'Rel. physical mem usage : ((MXBean(java.lang:type=OperatingSystem).FreePhysicalMemorySize / MXBean(java.lang:type=OperatingSystem).TotalPhysicalMemorySize))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS MarkSweep,type=GarbageCollector).CollectionTime))',
 'Rel. Swap Usage : ((\\Swap\\used / \\Swap\\total))' 
 ])

In [5]:
# data_test csvファイルの読み込み
test_df = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/test.zip (Unzipped Files)/test.csv',usecols=[
 'GC activity : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionCount))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionTime))',
 'Rel. swap usage : ((MXBean(java.lang:type=OperatingSystem).FreeSwapSpaceSize / MXBean(java.lang:type=OperatingSystem).TotalSwapSpaceSize))',
 'Stuck threads : (MXBean(com.bea:Name=ThreadPoolRuntime,Type=ThreadPoolRuntime).StuckThreadCount)',
 'Stuck threads : (MXBean(com.bea:ApplicationRuntime=sys01,Name=default,Type=WorkManagerRuntime).StuckThreadCount)',
 'Process CPU : (MXBean(java.lang:type=OperatingSystem).ProcessCpuLoad)',
 'Rel. heap committed : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.committed / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'System CPU : (MXBean(java.lang:type=OperatingSystem).SystemCpuLoad)',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Process CPU : (\\Process(java)\\CPU)',
 'Rel. heap usage : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.used / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'Rel. open file descriptors : ((MXBean(java.lang:type=OperatingSystem).OpenFileDescriptorCount / MXBean(java.lang:type=OperatingSystem).MaxFileDescriptorCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.committed / MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Daemon thread count : (MXBean(java.lang:type=Threading).DaemonThreadCount)',
 'host',
 'Total thread count : (MXBean(java.lang:type=Threading).ThreadCount)',
 'Rel. nonHeap usage : ((MXBean(java.lang:type=Memory).NonHeapMemoryUsage.used / MXBean(java.lang:type=Memory).NonHeapMemoryUsage.max))',
 'Rel. physical mem usage : ((MXBean(java.lang:type=OperatingSystem).FreePhysicalMemorySize / MXBean(java.lang:type=OperatingSystem).TotalPhysicalMemorySize))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS MarkSweep,type=GarbageCollector).CollectionTime))',
 'Rel. Swap Usage : ((\\Swap\\used / \\Swap\\total))' 
 ])

In [6]:
#スコア0.9279の予測データを読み込み
test_y = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/提出用ファイル/xgboost_trained_model_26_param_tuning_test_reloaded2.csv')

In [7]:
#目的変数のみを取り出し値を四捨五入
test_y=test_y["Anomaly"]
test_y=test_y.round(0).astype(int)

In [8]:
#訓練データから目的変数のみを取り出す
train_y=train_df["Anomaly"]

In [9]:
#訓練データから目的変数を除去
train_x=train_df.drop(["Anomaly"],axis=1)
test_x=test_df

In [10]:
#全データをpandasに適用
train_x = pd.DataFrame(train_x)
test_x = pd.DataFrame(test_x)
train_y = pd.Series(train_y)
test_y = pd.Series(test_y)

In [11]:
#trainデータとtestデータの予測値を結合
train2_x = pd.concat([train_x, test_x],axis=0)
train2_y = pd.concat([train_y, test_y],axis=0)

In [12]:
# 学習データにget_dummiesによるone-hot encodingを行う
train2_x = pd.get_dummies(train2_x, columns=["host"])
test_x = pd.get_dummies(test_x, columns=["host"])

In [13]:
reduce_mem_usage(train2_x, verbose=True)

Mem. usage decreased to 313.28 Mb (67.9% reduction)


,Daemon thread count : (MXBean(java.lang:type=Threading).DaemonThreadCount),"GC activity : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionCount))","GC time : (incld/dx (MXBean(java.lang:name=PS MarkSweep,type=GarbageCollector).CollectionTime))","GC time : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionTime))","Memory space usage : ((MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.committed / MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.max))","Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))","Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))","Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))","Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))","Memory space usage : ((MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.max))",...,host_lphost06,host_lphost07,host_lphost08,host_lphost09,host_lphost10,host_lphost11,host_lphost14,host_lphost15,host_lphost17,host_lphost18
0,62.0,0.0,0.0,0.0,0.927246,0.271729,0.165527,0.468262,0.382080,0.815918,...,1,0,0,0,0,0,0,0,0,0
1,62.0,0.0,0.0,0.0,0.927246,0.271729,0.178833,0.468262,0.382080,0.815918,...,1,0,0,0,0,0,0,0,0,0
2,62.0,0.0,0.0,0.0,0.927246,0.271729,0.196533,0.468262,0.382080,0.815918,...,1,0,0,0,0,0,0,0,0,0
3,62.0,0.0,0.0,0.0,0.927246,0.271729,0.209961,0.468262,0.382080,0.815918,...,1,0,0,0,0,0,0,0,0,0
4,62.0,0.0,0.0,0.0,0.927246,0.271729,0.222900,0.468262,0.382080,0.815918,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2927227,74.0,0.0,0.0,0.0,0.970215,0.099426,0.087524,0.265869,0.159668,0.532715,...,0,0,0,0,0,0,0,0,0,1
2927228,74.0,2.0,0.0,59.0,0.970215,0.095093,0.038757,0.265869,0.159668,0.532715,...,0,0,0,0,0,0,0,0,0,1
2927229,74.0,0.0,0.0,0.0,0.970215,0.095093,0.049927,0.265869,0.159668,0.532715,...,0,0,0,0,0,0,0,0,0,1
2927230,74.0,0.0,0.0,0.0,0.970215,0.095093,0.060730,0.265869,0.159668,0.532715,...,0,0,0,0,0,0,0,0,0,1


In [14]:
reduce_mem_usage(test_x, verbose=True)

Mem. usage decreased to 173.08 Mb (70.5% reduction)


,Daemon thread count : (MXBean(java.lang:type=Threading).DaemonThreadCount),"GC activity : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionCount))","GC time : (incld/dx (MXBean(java.lang:name=PS MarkSweep,type=GarbageCollector).CollectionTime))","GC time : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionTime))","Memory space usage : ((MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.committed / MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.max))","Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))","Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))","Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))","Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))","Memory space usage : ((MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.max))",...,host_lphost06,host_lphost07,host_lphost08,host_lphost09,host_lphost10,host_lphost11,host_lphost14,host_lphost15,host_lphost17,host_lphost18
0,66.0,0.0,0.0,0.0,0.971191,0.048828,0.036743,0.469238,0.254395,0.512207,...,1,0,0,0,0,0,0,0,0,0
1,66.0,1.0,0.0,25.0,0.971191,0.048401,0.004642,0.469238,0.254395,0.512207,...,1,0,0,0,0,0,0,0,0,0
2,66.0,0.0,0.0,0.0,0.971191,0.048401,0.019470,0.469238,0.254395,0.512207,...,1,0,0,0,0,0,0,0,0,0
3,66.0,0.0,0.0,0.0,0.971191,0.048401,0.034393,0.469238,0.254395,0.512207,...,1,0,0,0,0,0,0,0,0,0
4,66.0,1.0,0.0,26.0,0.971191,0.048492,0.004490,0.469238,0.254395,0.512207,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2927227,74.0,0.0,0.0,0.0,0.970215,0.099426,0.087524,0.265869,0.159668,0.532715,...,0,0,0,0,0,0,0,0,0,1
2927228,74.0,2.0,0.0,59.0,0.970215,0.095093,0.038757,0.265869,0.159668,0.532715,...,0,0,0,0,0,0,0,0,0,1
2927229,74.0,0.0,0.0,0.0,0.970215,0.095093,0.049927,0.265869,0.159668,0.532715,...,0,0,0,0,0,0,0,0,0,1
2927230,74.0,0.0,0.0,0.0,0.970215,0.095093,0.060730,0.265869,0.159668,0.532715,...,0,0,0,0,0,0,0,0,0,1


In [15]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train2_x))[0]
tr_x, va_x = train2_x.iloc[tr_idx], train2_x.iloc[va_idx]
tr_y, va_y = train2_y.iloc[tr_idx], train2_y.iloc[va_idx]


In [16]:
'''
# 学習データを学習データとバリデーションデータに分ける(数が多いのでもともとのvalデータを分割)
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(va_x))[0]
tr_x, va_x = train2_x.iloc[tr_idx], train2_x.iloc[va_idx]
tr_y, va_y = train2_y.iloc[tr_idx], train2_y.iloc[va_idx]
'''

'\n# 学習データを学習データとバリデーションデータに分ける(数が多いのでもともとのvalデータを分割)\nfrom sklearn.model_selection import KFold\n\nkf = KFold(n_splits=5, shuffle=True, random_state=71)\ntr_idx, va_idx = list(kf.split(va_x))[0]\ntr_x, va_x = train2_x.iloc[tr_idx], train2_x.iloc[va_idx]\ntr_y, va_y = train2_y.iloc[tr_idx], train2_y.iloc[va_idx]\n'

In [17]:
tr_x.shape

(3754218, 35)

In [18]:
va_x.shape

(938555, 35)

In [19]:
'''
# -----------------------------------
# ニューラルネットのパラメータチューニングの例
# -----------------------------------
from hyperopt import hp
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import ReLU, PReLU
from keras.layers.core import Dense, Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD,Adam
from sklearn.preprocessing import StandardScaler

# 基本となるパラメータ
base_param = {
    'input_dropout': 0.0,
    'hidden_layers': 3,
    'hidden_units': 96,
    'hidden_activation': 'relu',
    'hidden_dropout': 0.2,
    'batch_norm': 'before_act',
    'optimizer': {'type': 'adam', 'lr': 0.001},
    'batch_size': 64,
}

# 探索するパラメータの空間を指定する
param_space = {
    'input_dropout': hp.quniform('input_dropout', 0, 0.2, 0.05),
    'hidden_layers': hp.quniform('hidden_layers', 2, 4, 1),
    'hidden_units': hp.quniform('hidden_units', 32, 256, 32),
    'hidden_activation': hp.choice('hidden_activation', ['prelu', 'relu']),
    'hidden_dropout': hp.quniform('hidden_dropout', 0, 0.3, 0.05),
    'batch_norm': hp.choice('batch_norm', ['before_act', 'no']),
    'optimizer': hp.choice('optimizer',
                           [{'type': 'adam',
                             'lr': hp.loguniform('adam_lr', np.log(0.00001), np.log(0.01))},
                            {'type': 'sgd',
                             'lr': hp.loguniform('sgd_lr', np.log(0.00001), np.log(0.01))}]),
    'batch_size': hp.quniform('batch_size', 32, 128, 32),
}


class MLP:

    def __init__(self, params):
        self.params = params
        self.scaler = None
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):

        # パラメータ
        input_dropout = self.params['input_dropout']
        hidden_layers = int(self.params['hidden_layers'])
        hidden_units = int(self.params['hidden_units'])
        hidden_activation = self.params['hidden_activation']
        hidden_dropout = self.params['hidden_dropout']
        batch_norm = self.params['batch_norm']
        optimizer_type = self.params['optimizer']['type']
        optimizer_lr = self.params['optimizer']['lr']
        batch_size = int(self.params['batch_size'])

        # 標準化
        self.scaler = StandardScaler()
        tr_x = self.scaler.fit_transform(tr_x)
        va_x = self.scaler.transform(va_x)

        self.model = Sequential()

        # 入力層
        self.model.add(Dropout(input_dropout, input_shape=(tr_x.shape[1],)))

        # 中間層
        for i in range(hidden_layers):
            self.model.add(Dense(hidden_units))
            if batch_norm == 'before_act':
                self.model.add(BatchNormalization())
            if hidden_activation == 'prelu':
                self.model.add(PReLU())
            elif hidden_activation == 'relu':
                self.model.add(ReLU())
            else:
                raise NotImplementedError
            self.model.add(Dropout(hidden_dropout))

        # 出力層
        self.model.add(Dense(1, activation='sigmoid'))

        # オプティマイザ
        if optimizer_type == 'sgd':
            optimizer = SGD(lr=optimizer_lr, decay=1e-6, momentum=0.9, nesterov=True)
        elif optimizer_type == 'adam':
            optimizer = Adam(lr=optimizer_lr, beta_1=0.9, beta_2=0.999, decay=0.)
        else:
            raise NotImplementedError

        # 目的関数、評価指標などの設定
        self.model.compile(loss='binary_crossentropy',
                           optimizer=optimizer, metrics=['accuracy'])

        # エポック数、アーリーストッピング
        # あまりepochを大きくすると、小さい学習率のときに終わらないことがあるので注意
        nb_epoch = 3
        patience = 10
        early_stopping = EarlyStopping(patience=patience, restore_best_weights=True)

        # 学習の実行
        history = self.model.fit(tr_x, tr_y,
                                 epochs=nb_epoch,
                                 batch_size=batch_size, verbose=1,
                                 validation_data=(va_x, va_y),
                                 callbacks=[early_stopping])

    def predict(self, x):
        # 予測
        x = self.scaler.transform(x)
        y_pred = self.model.predict(x)
        y_pred = y_pred.flatten()
        return y_pred


# -----------------------------------
# パラメータチューニングの実行

from hyperopt import fmin, tpe, STATUS_OK, Trials
from sklearn.metrics import log_loss


def score(params):
    # パラメータセットを指定したときに最小化すべき関数を指定する
    # モデルのパラメータ探索においては、モデルにパラメータを指定して学習・予測させた場合のスコアとする
    model = MLP(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    print(f'params: {params}, logloss: {score:.4f}')

    # 情報を記録しておく
    history.append((params, score))

    return {'loss': score, 'status': STATUS_OK}


# hyperoptによるパラメータ探索の実行
max_evals = 20
trials = Trials()
history = []
fmin(score, param_space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

# 記録した情報からパラメータとスコアを出力する
# trialsからも情報が取得できるが、パラメータを取得しにくい
history = sorted(history, key=lambda tpl: tpl[1])
best = history[0]
print(f'best params:{best[0]}, score:{best[1]:.4f}')
'''

"\n# -----------------------------------\n# ニューラルネットのパラメータチューニングの例\n# -----------------------------------\nfrom hyperopt import hp\nfrom keras.callbacks import EarlyStopping\nfrom keras.layers.advanced_activations import ReLU, PReLU\nfrom keras.layers.core import Dense, Dropout\nfrom keras.layers import BatchNormalization\nfrom keras.models import Sequential\nfrom tensorflow.keras.optimizers import SGD,Adam\nfrom sklearn.preprocessing import StandardScaler\n\n# 基本となるパラメータ\nbase_param = {\n    'input_dropout': 0.0,\n    'hidden_layers': 3,\n    'hidden_units': 96,\n    'hidden_activation': 'relu',\n    'hidden_dropout': 0.2,\n    'batch_norm': 'before_act',\n    'optimizer': {'type': 'adam', 'lr': 0.001},\n    'batch_size': 64,\n}\n\n# 探索するパラメータの空間を指定する\nparam_space = {\n    'input_dropout': hp.quniform('input_dropout', 0, 0.2, 0.05),\n    'hidden_layers': hp.quniform('hidden_layers', 2, 4, 1),\n    'hidden_units': hp.quniform('hidden_units', 32, 256, 32),\n    'hidden_activation': hp.c

In [20]:
'''
# 記録した情報からパラメータとスコアを出力する
# trialsからも情報が取得できるが、パラメータを取得しにくい
history = sorted(history, key=lambda tpl: tpl[1])
best = history[0]
print(f'best params:{best[0]}, score:{best[1]:.4f}')
'''

"\n# 記録した情報からパラメータとスコアを出力する\n# trialsからも情報が取得できるが、パラメータを取得しにくい\nhistory = sorted(history, key=lambda tpl: tpl[1])\nbest = history[0]\nprint(f'best params:{best[0]}, score:{best[1]:.4f}')\n"

使用するパラメータ

{'batch_norm': 'no', 'batch_size': 32.0, 'hidden_activation': 'prelu', 'hidden_dropout': 0.2, 'hidden_layers': 3.0, 'hidden_units': 224.0, 'input_dropout': 0.0, 'optimizer': {'lr': 1.7231851937742915e-05, 'type': 'sgd'}

In [27]:
# tensorflowの警告抑制
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# -----------------------------------
# ニューラルネットの実装
# -----------------------------------
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import ReLU, PReLU
from keras.layers.core import Dense, Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD,Adam
from sklearn.preprocessing import StandardScaler

#ハイパーパラメータ
batch_norm = 'no', 
batch_size = 32
hidden_activation = 'PReLU', 
hidden_dropout = 0.2, 
hidden_layers = 3.0
hidden_units = 224 
input_dropout = 0.0,
optimizer_type = 'sgd',
optimizer_lr = 0.000017231851937742915
 

# データのスケーリング
scaler = StandardScaler()
tr_x = scaler.fit_transform(tr_x)
va_x = scaler.transform(va_x)
test_x = scaler.transform(test_x)

# ニューラルネットモデルの構築
model = Sequential()

model.add(Dense(224, activation='PReLU', input_shape=(tr_x.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(224, activation='PReLU'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

'''
# 入力層
model.add(Dropout(input_dropout, input_shape=(tr_x.shape[1],)))
model.add(Dropout(hidden_dropout))
# 中間層
model.add(Dense(hidden_units))
model.add(PReLU())
model.add(Dropout(hidden_dropout))

# 出力層
model.add(Dense(1, activation='sigmoid'))
'''

# オプティマイザ
optimizer = SGD(lr=optimizer_lr, decay=1e-6, momentum=0.9, nesterov=True)

# 目的関数、評価指標などの設定
model.compile(loss='binary_crossentropy',
                           optimizer=optimizer, metrics=['accuracy'])

# エポック数、アーリーストッピング
# あまりepochを大きくすると、小さい学習率のときに終わらないことがあるので注意
nb_epoch = 30
patience = 10
early_stopping = EarlyStopping(patience=patience, restore_best_weights=True)

# 学習の実行
history = model.fit(tr_x, tr_y,
                                 epochs=nb_epoch,
                                 batch_size=batch_size, verbose=1,
                                 validation_data=(va_x, va_y),
                                 callbacks=[early_stopping])

'''
model.add(Dense(256, activation='prelu', input_shape=(train2_x.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='prelu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

# アーリーストッピング
# -----------------------------------
from keras.callbacks import EarlyStopping

# アーリーストッピングの観察するroundを20とする
# restore_best_weightsを設定することで、最適なエポックでのモデルを使用する
epochs = 50
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(tr_x, tr_y,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(va_x, va_y), callbacks=[early_stopping])
'''

# バリデーションデータでのスコアの確認
va_pred = model.predict(va_x)
score = log_loss(va_y, va_pred, eps=1e-7)
print(f'logloss: {score:.4f}')

# 予測
pred = model.predict(test_x)

Epoch 1/30
117320/117320 [==============================] - 446s 4ms/step - loss: 0.0724 - accuracy: 0.9841 - val_loss: 0.0343 - val_accuracy: 0.9897
Epoch 2/30
117320/117320 [==============================] - 427s 4ms/step - loss: 0.0354 - accuracy: 0.9895 - val_loss: 0.0284 - val_accuracy: 0.9919
Epoch 3/30
117320/117320 [==============================] - 438s 4ms/step - loss: 0.0309 - accuracy: 0.9906 - val_loss: 0.0256 - val_accuracy: 0.9920
Epoch 4/30
117320/117320 [==============================] - 434s 4ms/step - loss: 0.0284 - accuracy: 0.9913 - val_loss: 0.0238 - val_accuracy: 0.9922
Epoch 5/30
117320/117320 [==============================] - 450s 4ms/step - loss: 0.0268 - accuracy: 0.9916 - val_loss: 0.0227 - val_accuracy: 0.9923
Epoch 6/30
117320/117320 [==============================] - 451s 4ms/step - loss: 0.0256 - accuracy: 0.9918 - val_loss: 0.0217 - val_accuracy: 0.9924
Epoch 7/30
117320/117320 [==============================] - 453s 4ms/step - loss: 0.0247 - accuracy:

In [28]:
# 構築したモデルの保存
import pickle
file = '/content/drive/MyDrive/Nishika ソフトウェアの異常検知/モデル/NN_26_param_tuning_test_reloaded.pkl'
pickle.dump(model, open(file, 'wb'))

In [29]:
# 構築したモデルの読み込み
import pickle
filename = '/content/drive/MyDrive/Nishika ソフトウェアの異常検知/モデル/NN_26_param_tuning_test_reloaded.pkl'
loaded_model = pickle.load(open(filename, 'rb'))

In [30]:
# submission csvファイルの読み込み
submission_df = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/data.zip (Unzipped Files)/sample_submission.csv')

In [31]:
id_df=submission_df.iloc[:,0]
id_df.head()

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

In [32]:
pred_df = pd.DataFrame(pred)
pred_df.head()

,0
0,0.000013
1,0.000012
2,0.000012
3,0.000013
4,0.000012


In [33]:
output_df=pd.concat([id_df, pred_df],axis=1)
output_df.columns=['id','Anomaly']
output_df.head()

,id,Anomaly
0,0,0.000013
1,1,0.000012
2,2,0.000012
3,3,0.000013
4,4,0.000012


In [34]:
output_df.to_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/提出用ファイル/NN_26_param_tuning_test_reloaded.csv',index=False)